In [1]:
from dotenv import load_dotenv
import os
from pprint import pprint

# trace langchain execution
os.environ["LANGCHAIN_TRACING"] = "false"

# load in keys from .env. Have in format API_KEY="".
load_dotenv()
API_KEY = os.getenv("API_KEY")
LOCATION = os.getenv("LOCATION")
GOOGLE_PROJECT_ID = os.getenv("GOOGLE_PROJECT_ID") # ie, confident-jackle-123456

In [2]:
from google.cloud import aiplatform

aiplatform.init(project=GOOGLE_PROJECT_ID, location="us-central1")

In [3]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import LLMChain, PromptTemplate
from langchain.llms import vertexai
from vertexai.preview.language_models import TextGenerationModel
from langchain.agents import AgentType, initialize_agent

In [4]:
import json

from tools import GetBasicMenuTool, FindItemIdTool, CreateOrderTool

## overview
In order to maintain customer privacy and minimize the ability for this agent to leak information, it is not going to have any memory (in the prompt). It is still going to be able to retrieve and relay a user's order, but that will be information retrieved from an api using a tool. 

This saves alot of tokens being fed into the input prompt

In [5]:
from square.client import Client
from square.api.catalog_api import CatalogApi
from pydantic import BaseModel, Field, ConfigDict
from langchain.tools.base import BaseTool
from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
client = Client(
    square_version='2023-08-16',
    access_token=API_KEY,
    environment='sandbox')

In [6]:
tools = [FindItemIdTool(), CreateOrderTool()]

In [7]:
agent = initialize_agent(
    tools,
    vertexai.VertexAI(temperature=0.3),
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [10]:
agent.run("Could I order a SMALL BLUEBERRY MUFFIN?")



> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "find_item_id_tool",
  "action_input": {
    "name": "SMALL BLUEBERRY MUFFIN"
  }
}
```

Observation: QHIFP5QQD373ZLO5P7FHGTCM
Thought: Action:
```
{
  "action": "create_new_order",
  "action_input": {
    "item_id": "QHIFP5QQD373ZLO5P7FHGTCM",
    "quantity": "1"
  }
}
```


Observation: <ApiResponse {"order": {"id": "wE0Q3Vxv0shfL4Xc4NSfXa9UXwPZY","location_id": "LETE3KSW0H1RB","line_items": [{"uid": "Yo2ppXqzSerrM8lbovOXKC","catalog_object_id": "QHIFP5QQD373ZLO5P7FHGTCM","catalog_version": 1694014352283,"quantity": "1","name": "Muffin","variation_name": "Small, Blueberry","base_price_money": {"amount": 300,"currency": "USD"},"gross_sales_money": {"amount": 300,"currency": "USD"},"total_tax_money": {"amount": 0,"currency": "USD"},"total_discount_money": {"amount": 0,"currency": "USD"},"total_money": {"amount": 300,"currency": "USD"},"variation_total_price_money": {"amount": 300,"currency": "USD"},"item_type": "ITEM","

'Your order for a small blueberry muffin has been placed.'